In [1]:
import json, pyld
from pyld import jsonld

In [45]:
institutions = json.load(open('../../MIP_institutions.json'))
consortiums = json.load(open('../../MIP_consortiums.json'))

In [4]:
institutions

{'institutions': {'AER': {'identifiers': {'institution_name': 'Atmospheric and Environmental Research',
    'aliases': [],
    'acronyms': ['AER'],
    'labels': [],
    'ror': '04cg70g73',
    'url': ['http://www.aer.com/'],
    'established': 1977,
    'type': 'Facility'},
   'location': {'lat': 42.539472,
    'lon': -71.256946,
    'city': 'Lexington',
    'country': ['United States', 'US']},
   'consortiums': []},
  'AS-RCEC': {'identifiers': {'institution_name': 'Research Center for Environmental Changes, Academia Sinica',
    'aliases': [],
    'acronyms': ['RCEC'],
    'labels': [],
    'ror': '050qpg053',
    'url': ['http://www.rcec.sinica.edu.tw/'],
    'established': None,
    'type': 'Facility'},
   'location': {'lat': 25.040712,
    'lon': 121.614171,
    'city': 'Taipei',
    'country': ['Taiwan', 'TW']},
   'consortiums': []},
  'AWI': {'identifiers': {'institution_name': 'Alfred Wegener Institute for Polar and Marine Research',
    'aliases': [],
    'acronyms': ['AWI']

In [11]:
item = list(institutions['institutions'].items())[0]
item

('AER',
 {'identifiers': {'institution_name': 'Atmospheric and Environmental Research',
   'aliases': [],
   'acronyms': ['AER'],
   'labels': [],
   'ror': '04cg70g73',
   'url': ['http://www.aer.com/'],
   'established': 1977,
   'type': 'Facility'},
  'location': {'lat': 42.539472,
   'lon': -71.256946,
   'city': 'Lexington',
   'country': ['United States', 'US']},
  'consortiums': []})

In [38]:

def parse_institution (item):

    val = item[1]['identifiers']
    contents = []

    updated  = {
        "@id": f'mip-cmor-tables/institutions/{item[0].lower()}',
        "@type": "cmip:institution",
    }

    updated['institution:cmip_acronym'] = item[0]
    updated['institution:ror'] = val['ror']
    updated['institution:name'] = val['institution_name']

    for i in 'url established type labels aliases acronyms'.split(): 
        updated["institution:"+i] = val[i]


    updated['institution:location'] = {
        "@id": f"mip-cmor-tables/institutions/location/{val['ror']}",
        "@type": "institution:location"
        }



    newloc = {}
    for i in item[1]['location']:
        newloc['location:'+i] = item[1]['location'][i]
    

    updated['institution:location'].update({"@nest": newloc})
    return updated




In [39]:
import tqdm 
import json 
for item in tqdm.tqdm(institutions['institutions'].items()):
    data = parse_institution(item)
    json.dump(data,open(f'./institutions/{item[0]}.json','w'),indent=4)
    
    


100%|██████████| 56/56 [00:00<00:00, 100.25it/s]


In [46]:
inst = [i.lower() for i in institutions['institutions'].keys()]


{'consortiums': {'CNRM-CERFACS': {'name': 'CNRM-CERFACS',
   'contains': ['CNRM (Centre National de Recherches Meteorologiques, Toulouse 31057, France)',
    'CERFACS (Centre Europeen de Recherche et de Formation Avancee en Calcul Scientifique, Toulouse 31057, France)']},
  'CNRM-Cerfacs': {'name': 'CNRM-Cerfacs',
   'contains': ['CNRM (Centre National de Recherches Meteorologiques, Toulouse 31057, France), CERFACS (Centre Europeen de Recherche et de Formation Avancee en Calcul Scientifique, Toulouse 31100, France)']},
  'CSIRO-ARCCSS': {'name': 'CSIRO-ARCCSS',
   'contains': ['CSIRO (Commonwealth Scientific and Industrial Research Organisation, Aspendale, Victoria 3195, Australia)',
    'ARCCSS (Australian Research Council Centre of Excellence for Climate System Science)']},
  'CSIRO-COSIMA': {'name': 'CSIRO-COSIMA',
   'contains': ['CSIRO (Commonwealth Scientific and Industrial Research Organisation, Australia)',
    'COSIMA (Consortium for Ocean-Sea Ice Modelling in Australia)']},
 

In [57]:

missing = {}

for i,j in tqdm.tqdm(consortiums['consortiums'].items()):
    
    
    shortmiss = []
    members = []
    for k in j['contains']:
        l = k.split()[0].lower()
        if l in inst:
            members.append({"@id": f'mip-cmor-tables/institutions/{l}'})
        else:
            members.append({"@id": f'{k}'})
            shortmiss.append(k)
            
    missing[i] = shortmiss
    
    
    entry = {
        "@id": f'mip-cmor-tables/consortia/{i.lower()}',
        "@type": "cmip:consortium",
        "consortium:cmip_acronym":i,
        "consortium:description":j['name'],
        "consortium:url":"",
        "consortium:changes":"",
        "consortium:members":{
            "@set": members,
        }
        
    }
    
    
    json.dump(entry,open(f'./consortia/{i.lower()}.json','w'),indent=4)
    
print(missing)
with open('./consortia/missing.txt','w') as f:
    json.dump(missing,f,indent=4)

100%|██████████| 21/21 [00:00<00:00, 1769.29it/s]

{'CNRM-CERFACS': ['CNRM (Centre National de Recherches Meteorologiques, Toulouse 31057, France)', 'CERFACS (Centre Europeen de Recherche et de Formation Avancee en Calcul Scientifique, Toulouse 31057, France)'], 'CNRM-Cerfacs': ['CNRM (Centre National de Recherches Meteorologiques, Toulouse 31057, France), CERFACS (Centre Europeen de Recherche et de Formation Avancee en Calcul Scientifique, Toulouse 31100, France)'], 'CSIRO-ARCCSS': ['ARCCSS (Australian Research Council Centre of Excellence for Climate System Science)'], 'CSIRO-COSIMA': ['COSIMA (Consortium for Ocean-Sea Ice Modelling in Australia)'], 'DLR-BIRA': ['Deutsches Zentrum fur Luft- und Raumfahrt', 'Royal Belgian Institute for Space Aeronomy'], 'E3SM-Project': ['ANL (Argonne National Laboratory, Argonne, IL 60439, USA)', 'BNL (Brookhaven National Laboratory, Upton, NY 11973, USA)', 'LANL (Los Alamos National Laboratory, Los Alamos, NM 87545, USA)', 'LBNL (Lawrence Berkeley National Laboratory, Berkeley, CA 94720, USA)', 'ORNL

In [48]:
entry


{'@id': 'mip-cmor-tables/consortium/cnrm-cerfacs',
 '@type': 'cmip:consortium',
 'consortium:cmip_acronym': 'CNRM-CERFACS',
 'consortium:description': 'CNRM-CERFACS',
 'consortium:url': '',
 'consortium:changes': '',
 'consortium:members': {'@set': [{'@id': 'CNRM (Centre National de Recherches Meteorologiques, Toulouse 31057, France)'},
   {'@id': 'CERFACS (Centre Europeen de Recherche et de Formation Avancee en Calcul Scientifique, Toulouse 31057, France)'}]}}

In [51]:
'pnnl' in inst

False